___
# __Convolutional Neural Network from scratch__
### _Author: Aki Taniguchi_
### _Original date: 19/02/2020_
### _Last update: 28/02/2020_
___

## I. Setting environment
___

In [1]:
import sys
sys.path.append("C:\\Users\\tngch\\Python Code and AI\\Neural Network")

# Load libraries
import numpy as np
from Deep_Neural_Network import *

In [2]:
X = np.random.rand(3, 32, 32, 10)
Y = np.random.choice(2, 10)

A = {}
A['0'] = X

In [3]:
# Setting up the hyper-parameters ("LeNet-5")
architecture = ['init', 'conv', 'maxpool', 'conv', 'maxpool', 'fc', 'fc']
activations = ['init', 'relu', 'relu', 'relu', 'relu', 'relu', 'sigmoid'] # Pooling layer has no activation, but we need to specify the same as convolutional layer for backpropagation calculation purposes
filter_size = [0, 5, 2, 5, 2, 1, 1]
nb_kernel = [3, 8, 8, 16, 16, 120, 84] # first is RGB, fully connected layers are the number of neurons
padding = [0, 0, 0, 0, 0, 0, 0]
stride = [0, 1, 2, 1, 2, 0, 0]
L = len(architecture)

## II. Initialize model
___

In [4]:
# Define Shape of Output A. Note that the layer with full connection needs to be vectorized, but won't be here
# dim A is (channel, height, width, observation)
# width and height are both determined: int[(x + 2p - f) / s + 1]
def get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A):

    layer_shape = [A['0'].shape]

    m = A['0'].shape[3]
    n_h = A['0'].shape[1]
    n_w = A['0'].shape[2]
    
    L = len(architecture)
    f = filter_size
    k = nb_kernel
    p = padding
    s = stride

    # Note the last layer has only 2 outcome as we are not doing a softmax. This needs to be changed once it will be implemented.
    for l in range(1, L+1):
        if l != L:
            if architecture[l] != 'fc':
                n_h = int((n_h + 2*p[l] - f[l]) / s[l] + 1)
                n_w = int((n_w + 2*p[l] - f[l]) / s[l] + 1)
                layer_shape.append((k[l], n_h, n_w, m))
            else:
                layer_shape.append((k[l], m))
        else:
            layer_shape.append((2, m))

    return layer_shape

In [5]:
# Test get_layer_shape
layer_shape = get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A)

layer_shape

[(3, 32, 32, 10),
 (8, 28, 28, 10),
 (8, 14, 14, 10),
 (16, 10, 10, 10),
 (16, 5, 5, 10),
 (120, 10),
 (84, 10),
 (2, 10)]

In [6]:
# Parameters size:
# If Conv/Pool: (curr channel, prev channel, filter, filter) and (cur channel, 1, 1, 1)
# If FC from Conv/Pool: (curr channel, vectorized[prev output])
# If FC from FC: (curr channel, prev channel)
# bias is always (curr channel, 1) when FC
def initialize_model(architecture, filter_size, nb_kernel, padding, stride, A):

    W = {}; b = {}; Z = {}; dA = {}; dZ = {}; dW = {}; db = {}
    L = len(architecture)
    f = filter_size
    k = nb_kernel
    m = A['0'].shape[3]
    layer_shape = get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A)

    for l in range(1, L):

        # We need to initialize output as well to allocate the convolution output per index (otherwise Python doesn't allow allocation)
        A[str(l)] = np.zeros((layer_shape[l]))
        dA[str(l)] = np.zeros((layer_shape[l]))

        # Now initializing parameters
        if architecture[l] == 'conv':
            W[str(l)] = np.random.randn(k[l], k[l-1], f[l], f[l]) * 0.01
            b[str(l)] = np.zeros((k[l], 1, 1, 1))
            Z[str(l)] = np.zeros((layer_shape[l]))
            dZ[str(l)] = np.zeros((layer_shape[l]))

        # Note that there is no parameter for Pooling layers (there is no Z neither, but we need to keep it in order to compute the generalized backpropagation)
        elif (architecture[l] == 'maxpool') | (architecture[l] == 'avgpool'):
            W[str(l)] = 0
            b[str(l)] = 0
            Z[str(l)] = 0
            dZ[str(l)] = 0

            # We specifically need to raise error if pooling layer kernel is different from convolutional layer kernel
            if k[l] != k[l-1]:
                raise ValueError("Pooling layer kernel # needs to be equal to Convolutional layer kernel #")

        elif architecture[l] =='fc':
            # Parameters size different at the moment of change from conv to fc due to vectorized output
            Z[str(l)] = np.zeros((layer_shape[l]))
            dZ[str(l)] = np.zeros((layer_shape[l]))
            
            if architecture[l-1] != 'fc':
                W[str(l)] = np.random.randn(k[l], int(np.prod(A[str(l-1)].shape) / m))
                dA[str(l-1)] = np.zeros((int(np.prod(A[str(l-1)].shape) / m), m))
            
            else:
                W[str(l)] = np.random.randn(k[l], k[l-1])
            
            b[str(l)] = np.zeros((k[l], 1))
        
        dW[str(l)] = W[str(l)]
        db[str(l)] = b[str(l)]

    return A, Z, W, b, dA, dZ, dW, db

In [7]:
# Test initialize_model
A, Z, W, b, dA, dZ, dW, db = initialize_model(architecture, filter_size, nb_kernel, padding, stride, A)

for l in range(L):
    print(architecture[l])
    print("A[{0}] shape: {1}".format(l, A[str(l)].shape))
    if l != 0:
        if (architecture[l] == 'maxpool') | (architecture[l] == 'avgpool'):
            print("Z[{0}] shape: {1}".format(l, Z[str(l)]))
            print("W[{0}] shape: {1}".format(l, W[str(l)]))
            print("b[{0}] shape: {1}".format(l, b[str(l)]))
            print("dW[{0}] shape: {1}".format(l, dW[str(l)]))
            print("db[{0}] shape: {1}".format(l, db[str(l)])) 
            print("dZ[{0}] shape: {1}".format(l, dZ[str(l)]))         
        else:
            print("Z[{0}] shape: {1}".format(l, Z[str(l)].shape))
            print("W[{0}] shape: {1}".format(l, W[str(l)].shape))
            print("b[{0}] shape: {1}".format(l, b[str(l)].shape))
            print("dW[{0}] shape: {1}".format(l, dW[str(l)].shape))
            print("db[{0}] shape: {1}".format(l, db[str(l)].shape)) 
            print("dZ[{0}] shape: {1}".format(l, dZ[str(l)].shape))
        
        print("dA[{0}] shape: {1}".format(l, dA[str(l)].shape)) 

init
A[0] shape: (3, 32, 32, 10)
conv
A[1] shape: (8, 28, 28, 10)
Z[1] shape: (8, 28, 28, 10)
W[1] shape: (8, 3, 5, 5)
b[1] shape: (8, 1, 1, 1)
dW[1] shape: (8, 3, 5, 5)
db[1] shape: (8, 1, 1, 1)
dZ[1] shape: (8, 28, 28, 10)
dA[1] shape: (8, 28, 28, 10)
maxpool
A[2] shape: (8, 14, 14, 10)
Z[2] shape: 0
W[2] shape: 0
b[2] shape: 0
dW[2] shape: 0
db[2] shape: 0
dZ[2] shape: 0
dA[2] shape: (8, 14, 14, 10)
conv
A[3] shape: (16, 10, 10, 10)
Z[3] shape: (16, 10, 10, 10)
W[3] shape: (16, 8, 5, 5)
b[3] shape: (16, 1, 1, 1)
dW[3] shape: (16, 8, 5, 5)
db[3] shape: (16, 1, 1, 1)
dZ[3] shape: (16, 10, 10, 10)
dA[3] shape: (16, 10, 10, 10)
maxpool
A[4] shape: (16, 5, 5, 10)
Z[4] shape: 0
W[4] shape: 0
b[4] shape: 0
dW[4] shape: 0
db[4] shape: 0
dZ[4] shape: 0
dA[4] shape: (400, 10)
fc
A[5] shape: (120, 10)
Z[5] shape: (120, 10)
W[5] shape: (120, 400)
b[5] shape: (120, 1)
dW[5] shape: (120, 400)
db[5] shape: (120, 1)
dZ[5] shape: (120, 10)
dA[5] shape: (120, 10)
fc
A[6] shape: (84, 10)
Z[6] shape: (

## III. Creating helper functions
___
Padding  
Slicing  
Vectorization

In [8]:
# We won't be using numpy's pad function as this one is enough and quick to operate
# Only works for 3D matrix
def add_padding(A, padding, value=0, axis=(0, 1, 2)):

    kernel, height, width = axis

    horizontal_pad = np.zeros((A.shape[kernel], padding, A.shape[width] + 2*padding)) + value
    vertical_pad = np.zeros((A.shape[kernel], A.shape[height], padding)) + value

    padded_A = np.concatenate((vertical_pad, A), axis=width)
    padded_A = np.concatenate((padded_A, vertical_pad), axis=width)
    padded_A = np.concatenate((horizontal_pad, padded_A), axis=height)
    padded_A = np.concatenate((padded_A, horizontal_pad), axis=height)

    return padded_A

In [9]:
# Test add_padding
pad = 1
padded_matrix = add_padding(A['0'][:,:,:,0], pad, value=0, axis=(0, 1, 2))

print("Padding:", pad)
print("Original matrix shape:", A['0'][:,:,:,0].shape)
print("Padded matrix:", padded_matrix.shape)
print("___________________")
print("Original matrix:", A['0'][:,:,:,0])
print("___________________")
print("Padded matrix:", padded_matrix)

Padding: 1
Original matrix shape: (3, 32, 32)
Padded matrix: (3, 34, 34)
___________________
Original matrix: [[[0.37151965 0.07025497 0.31861885 ... 0.72898616 0.02767313 0.13290114]
  [0.62471469 0.77584161 0.05978884 ... 0.75453208 0.49078958 0.99886057]
  [0.0302718  0.61844605 0.11690156 ... 0.10171411 0.14334798 0.25977845]
  ...
  [0.0042432  0.08342377 0.79400455 ... 0.98278525 0.17494768 0.80045494]
  [0.19861717 0.02868305 0.54998394 ... 0.18053927 0.24176602 0.15441161]
  [0.21969244 0.84510994 0.95508207 ... 0.92476469 0.52470126 0.37176531]]

 [[0.09710171 0.47013497 0.72139299 ... 0.75536227 0.05757846 0.89362501]
  [0.98098903 0.48172223 0.52671822 ... 0.39910069 0.6457488  0.61975799]
  [0.26151976 0.80411867 0.49233817 ... 0.02958888 0.93669982 0.29381764]
  ...
  [0.79067032 0.41422091 0.17277068 ... 0.17401158 0.1624773  0.37149736]
  [0.30883119 0.06104547 0.17586591 ... 0.4912697  0.48825867 0.74583437]
  [0.59259736 0.80382458 0.22061806 ... 0.85656882 0.87032323 

In [10]:
# Helper function for vectorization / devectorization
def vectorization(original_matrix, matrix_dict, architecture, layer, type='vector'):

    # Vectorization
    if type == 'vector':
        if (architecture[layer] == 'fc') & (architecture[layer-1] != 'fc'):
            # When the layer # is 'Fully Connected', we need to vectorize the previous output
            # or pass the previous output if it is not 'fc' as no need to do any vectorization
            # This does not work for Z, W, or b with the very first hidden layer (only A) as they do not have any value assigned 
            dim_1, dim_2, dim_3, dim_4 = matrix_dict[str(layer-1)].shape
            new_val = original_matrix.reshape(dim_1*dim_2*dim_3, dim_4)
        else:
            new_val = original_matrix

    # Devectorization
    elif type == 'matrix':
        if (architecture[layer+1] == 'fc') & (architecture[layer] != 'fc'):
            new_val = original_matrix.reshape(matrix_dict[str(layer)].shape)
        else:
            new_val = original_matrix
    else:
        raise ValueError("Type unknown: must be 'vector' or 'matrix'")

    return new_val

In [11]:
# Test vectorization function
for l in range(1, L):
    print("Layer:", l, architecture[l])

# Vectorization test function
def vec_test(l, type):
    print("\n_____________")
    if type == 'vector':
        print(architecture[l-1], architecture[l])
        print('Fwd Original shape:', A[str(l-1)].shape)
        print("Vectorized shape:", vectorization(A[str(l-1)], A, architecture, l, type).shape)
    if type == 'matrix':
        print(architecture[l+1], architecture[l])
        print('Bckwd Original shape:', dA[str(l+1)].shape)
        print("Devectorized shape:", vectorization(dA[str(l+1)], A, architecture, l+1, type).shape)

# Case where we need to vectorize. We are in the "fully connected layer" (l=5), and we use vectorization on the previous layer to do the forward propagation
vec_test(5, 'vector')

# Case where we need to devectorize. We are in the layer before the "fully connected" layer (e.g. one step after the fully connected layer backward, l=4), but we need to devectorize the "fully connected" layer output gradient to keep going backward
vec_test(3, 'matrix')

# We now make sure that no vectorization happens when it is not necessary. Let us test vectorization on layer 2 and layer 4.
vec_test(2, 'vector')
vec_test(4, 'matrix')

Layer: 1 conv
Layer: 2 maxpool
Layer: 3 conv
Layer: 4 maxpool
Layer: 5 fc
Layer: 6 fc

_____________
maxpool fc
Fwd Original shape: (16, 5, 5, 10)
Vectorized shape: (400, 10)

_____________
maxpool conv
Bckwd Original shape: (400, 10)
Devectorized shape: (16, 5, 5, 10)

_____________
conv maxpool
Fwd Original shape: (8, 28, 28, 10)
Vectorized shape: (8, 28, 28, 10)

_____________
fc maxpool
Bckwd Original shape: (120, 10)
Devectorized shape: (120, 10)


In [12]:
# Note this only works for a 4D matrix
def slicing(A, dA, layer, architecture, filter_size, padding, stride, type='forward'):

    # Padding the matrix to work the slicing on
    # This is base matrix where we are going to create all the necessary slicing (hence the need to pad it first)
    # Also note that we don't need to specify the value of kernel, as we won't slice through the depth (instead we take it all)
    # It takes as input a 4D matrix (for all observation), but return slice for only one observation
    f = filter_size
    s = stride

    # For backpropagation, the slice is bound by the dimension of the previous gradient shape
    # Whereas forward propagation slice is bound by the dimension of the current output shape
    if type == 'forward':
        kernel, height, width, observation = A[str(layer)].shape
        back = 0
    elif type == 'backward':
        kernel, height, width, observation = vectorization(A[str(layer+1)], dA, architecture, layer+1, 'matrix').shape
        back = 1
    else:
        raise ValueError("Type must be 'forward' or 'backward'")

    for m in range(observation):
        padded_A = add_padding(A[str(layer-1 + back)][:, :, :, m], padding[layer], value=0, axis=(0, 1, 2))
        # +1 is added for back propagation, as it works on the current gradient
        # Forward propagation works on the previous output

        for i in range(height):
            v1 = i * s[layer]
            v2 = i * s[layer] + f[layer]

            for j in range(width):
                h1 = j * s[layer]
                h2 = j * s[layer] + f[layer]

                # If convolutional layer, we need to return a fxfxkernel 3D matrix
                # Note that backpropagation is one step ahead, therefore we need to add +1 to the index
                if architecture[layer + back] == 'conv':
                    slice = padded_A[:, v1:v2, h1:h2]
                    # We don't exactly need k, but we still return it to yield the exact number of output in both cases
                    k = 0

                    yield slice, i, j, k, m

                # If pooling layer, we need an output slice of a fxf 2D matrix for each l-1 kernel (we have set kernel to be (l) given that (l-1) = (l) kernel)
                elif (architecture[layer + back] == 'maxpool') | (architecture[layer + back] == 'avgpool'):
                    for k in range(kernel):
                        slice = padded_A[k, v1:v2, h1:h2]

                        yield slice, i, j, k, m

In [13]:
# Test slicing
layer = 2; cnt = 0; dim_check = 0

for slice, i, j, k, m in slicing(A, dA, layer, architecture, filter_size, padding, stride, type='forward'):
    if (i < 2) & (j < 2) & (k==0) & (m==0):
        print(i, j, k, m, slice.shape, slice)
    cnt += 1

if architecture[layer] == 'conv':
    dim_check = np.prod(A[str(layer)].shape) / A[str(layer)].shape[0]
    theoretical_shape = (nb_kernel[layer-1], filter_size[layer], filter_size[layer])
elif (architecture[layer] == 'maxpool') | (architecture[layer] == 'avgpool'):
    dim_check = np.prod(A[str(layer)].shape)
    theoretical_shape = (filter_size[layer], filter_size[layer])

print("\n___________________")
print("Current layer architecture:", architecture[layer])
print("# of slices:", cnt)
print("Theoretical # of slices:", dim_check)
print("Theoretical dimension of slice:", theoretical_shape)
print("Real dimenion of slice:", slice.shape)

0 0 0 0 (2, 2) [[0. 0.]
 [0. 0.]]
0 1 0 0 (2, 2) [[0. 0.]
 [0. 0.]]
1 0 0 0 (2, 2) [[0. 0.]
 [0. 0.]]
1 1 0 0 (2, 2) [[0. 0.]
 [0. 0.]]

___________________
Current layer architecture: maxpool
# of slices: 15680
Theoretical # of slices: 15680
Theoretical dimension of slice: (2, 2)
Real dimenion of slice: (2, 2)


## IV. Convolutional Forward Propagation
___
Convolutional Layer:
$$ Z ^{[l]} _{(c, i, j, m)} = \sum _{k} ^{K} \sum _{h=1} ^{f^{[l]}_h} \sum_{w=1} ^{f^{[l]}_w} W^{[l]} _{(c, k, h, w)} \times \operatorname{pad} (A^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w), m)} + b^{[l]} _{(c)} \tag{1} $$  
$$ A^{[l]} _{(c, i, j, m)} = g(Z ^{[l]} _{(c, i, j, m)}) \tag{2} $$  
With: $i = h^{[l]} = \lfloor \frac {h^{[l-1]} + 2p^{[l]} - f^{[l]} _h} {s^{[l]}} \rfloor + 1 $, $j = w^{[l]} = \lfloor \frac {w^{[l-1]} + 2p^{[l]} - f^{[l]} _w} {s^{[l]}} \rfloor + 1 $  

Here, $h^{[l]}$ and $w^{[l]}$ (therefore i and j) are the height and width of layer \[l], $p^{[l]}$ is the padding added to the output \[l-1], $s^{[l]}$ is the stride layer l, c is the channel of layer \[l], K is the channel of layer \[l-1], $f^{[l]}_h$ and $f^{[l]}_w$ the height and width of the filter of layer \[l], and lastly m the number of observation.    
Example, i = 1, j = 2...

Pooling is to reduce size, and makes the computation easier. There is also an L2 pooling in order to control overfitting.  
Max Pooling Layer:  
$$ A ^{[l]} _{(c, i, j, m)} = \max {( A^{[l-1]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)} )} \tag{3} $$  
Average Pooling Layer:  
$$ A ^{[l]} _{(c, i, j, m)} = \frac {1} {f^{[l]}_h \times f^{[l]}_w} \times \sum _{h=1} ^{f^{[l]}_h} \sum_{w=1} ^{f^{[l]}_w} A^{[l-1]} _{(s(i-1)+h, s(j-1)+w, m)} \tag{4} $$    

With: $h_{1} = s(i-1) + 1$, $h_{2} = s(i-1) + f^{[l]}_h$, $w_{1} = s(j-1) + 1$, $w_{2} = s(j-1) + f^{[l]}_w$, i and j is calculated the same way as convolutional layer. In this layer, there is no $Z^{[l]}$, and the number of channel is equivalent to the previous layer.  

Vectorization to Fully connected Layer:  
In order to pass the output to the fully connected layer, we need to reduce the dimension e.g. vectorize the output so that it becomes from size (k, h, w, m) to (k*h*w, m), 4D ==> 2D. We will call the vectorization function as f, such as:  
$$ f(A^{[l]} _{(k^{[l]}, h^{[l]}, w^{[l]}, m)}) = A^{[l]} _{(k^{[l]} \times h^{[l]} \times w^{[l]}, m)} \tag{5} $$  

Fully connected Layer:  
Once the vectorization is done, we can then compute the output of the fully connected layer (e.g. traditional Neural Network calculation):  
$$Z^{[l]} = W^{[l]} \times f(A^{[l-1]}) + b^{[l]} \tag{6} $$
$$A^{[l]} = g(Z^{[l]}) \tag{7} $$

In [14]:
def convolutional_forward_propagation(A, Z, W, b, architecture, activations, filter_size, nb_kernel, padding, stride):
    
    L = len(architecture)

    # Through each layer
    for l in range(1, L):
            
        # We operate the convolution nb kernel time (corresponding to the output layer kernel, or the current layer filter kernel size)
        if architecture[l] == 'conv':
            # As mentionned in slicing, loop over kernel is different for pooling and conv (pooling is already in slicing, vs conv is looping output kernel size time)
            observation = A[str(l)].shape[0]
            for k in range(observation):
                for slice, i, j, _, m in slicing(A, dA, l, architecture, filter_size, padding, stride, type='forward'):
                    # We can see that the formula is highly similar to the usual forward propagation, the only diff is that it is an element-wise
                    # multiplication instead of a matrix multiplication, and we should return a scalar only
                    Z[str(l)][k, i, j, m] = np.sum(np.multiply(W[str(l)][k], slice) + b[str(l)][k])
                    A[str(l)][k, i, j, m] = activation_function(activations[l], Z[str(l)][k, i, j, m])

        elif architecture[l] == 'maxpool':
            for slice, i, j, k, m in slicing(A, dA, l, architecture, filter_size, padding, stride, type='forward'):
                A[str(l)][k, i, j, m] = np.max(slice)

        elif architecture[l] == 'avgpool':
            for slice, i, j, k, m in slicing(A, dA, l, architecture, filter_size, padding, stride, type='forward'):
                A[str(l)][k, i, j, m] = np.mean(slice)

        elif architecture[l] == 'fc':
            Z[str(l)] = np.dot(W[str(l)], vectorization(A[str(l-1)], A, architecture, l, type='vector')) + b[str(l)]
            A[str(l)] = activation_function(activations[l], Z[str(l)])

    return Z, A

In [15]:
# Test forward prop
Z, A = convolutional_forward_propagation(A, Z, W, b, architecture, activations, filter_size, nb_kernel, padding, stride)

for l in range(L):
    print("Layer {0}, Shape {1}".format(l, A[str(l)].shape))

print("Last layer value:", A[str(L-1)])
print("Cost:", compute_cost(A[str(L-1)], Y))

Layer 0, Shape (3, 32, 32, 10)
Layer 1, Shape (8, 28, 28, 10)
Layer 2, Shape (8, 14, 14, 10)
Layer 3, Shape (16, 10, 10, 10)
Layer 4, Shape (16, 5, 5, 10)
Layer 5, Shape (120, 10)
Layer 6, Shape (84, 10)
Last layer value: [[0.37966676 0.37643932 0.35895938 0.3381942  0.30435635 0.38299862
  0.3159191  0.3612046  0.35247208 0.3336124 ]
 [0.5490493  0.66401687 0.53471855 0.6238223  0.65146576 0.66551313
  0.68189881 0.57888066 0.6572134  0.69305806]
 [0.34918772 0.24539039 0.41191404 0.25217365 0.28468065 0.3533122
  0.3676646  0.34483278 0.26534948 0.30586712]
 [0.70185093 0.73262428 0.63274314 0.610737   0.60904756 0.66787278
  0.63483771 0.69259392 0.69830401 0.6226203 ]
 [0.43908671 0.36580498 0.38033638 0.35741605 0.46378682 0.3963641
  0.37060211 0.33772418 0.44423355 0.3551965 ]
 [0.63912454 0.6316727  0.66731639 0.62456743 0.56633069 0.57387684
  0.61174786 0.64180271 0.55750235 0.5925119 ]
 [0.63910396 0.59167442 0.6202843  0.62769592 0.5889897  0.64148359
  0.5476277  0.5583611

## V. Convolutional Backward Propagation
___
Recall the generalized scheme: Conv ==> Pool ==> Vectorized ==> FC.  
Derivation is therefore: FC ==> Devectorized ==> Pool ==> Conv.
  
Fully connected layer:  
We simply reuse the traditional fully connected neural network formulas:
$$dA^{[L]} = \frac{\partial Cost}{\partial A^{[L]}} = \frac{-Y}{A^{[L]}} + \frac{(1-Y)}{(1-A^{[L]})} \tag{8}$$
$$dZ^{[L]} = \frac{\partial Cost}{\partial A^{[L]}} \frac{\partial A^{[L]}}{\partial Z^{[L]}} = dA^{[L]} * g'(Z^{[L]}) \tag{9}$$

If there would have only one fully connected layer, we would stop at equation (9) and get gradients for parameters W and b.
$$dW^{[l]} = \frac{1}{m} dZ^{[l]} \times A'^{[l-1]} \tag{10}$$  
$$db^{[l]} = \frac{1}{m} \sum \limits_{i=1}^{m} {dZ^{[l]}} \tag{11}$$  

For more fully connected layers at the end, it goes:
$$dA^{[l-1]} = W'^{[l]} \times dZ^{[l]} \tag{10}$$
$$dZ^{[l-1]} = W'^{[l]} \times dZ^{[l]} * g'(Z^{[l-1]}) \tag{11}$$  
And we use (10) and (11) for dW, db of layer \[l-1].  
  
  
Devectorization:  
Recall that the first fully connected layer input is the vectorized \[l-1] output, e.g. assuming the vectorization is done at layer \[l]: $Z^{[l]} = W^{[l]} \times f(A^{[l-1]}) + b^{[l]}$. This means that we need to calculate $\frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial f(A^{[l-1]})}$ first.  
$$ dA^{[l-1]} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial f(A^{[l-1]})} = W'^{[l]} \times dZ^{[l]}$$  
And now we can devectorize:  
$$ \operatorname{devectorized} (dA^{[l-1]}) = F^{-1}(\frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}}{\partial f(A^{[l-1]})}) = F^{-1}(W'^{[l]} \times dZ^{[l]}) \tag{12} $$
Which is of dimension $(k^{[l-1]}, h^{[l-1]}, w^{[l-1]}, m)$. We can now pass on the devectorized gradient to the following layers.
  
Pooling layer:  
Pooling layers have no parameters, but we still need to instruct the following layers of the "winner cells" to continue the backward propagation process. In each case, we have: $h_{1} = s(i-1) + 1$, $h_{2} = s(i-1) + f^{[l]}_h$, $w_{1} = s(j-1) + 1$, $w_{2} = s(j-1) + f^{[l]}_w$, i and j is calculated the same way as convolutional layer, and dimension of the output is the same as the dimension of the input. Note that in both case, $dA^{[l]}$ is initiated with 0, and the value of backpropagation is added to each cell (e.g. we stack up its value when cells are overlapping). 

Max Pooling:
For Max pooling, winner cells are the one which are the maximum of the previous layer's output. The rest are equals to 0. Therefore:  
$$ dA^{[l]} _{(c, h, w, m)} = \frac{\partial Cost}{\partial f(A^{[l]})} \frac{\partial f(A^{[l]})}{\partial A^{[l]}} = dA^{[l]} _{(c, h, w, m)} + \max (\operatorname{devectorized} (dA^{[l+1]} _{(c, h_{1}:h_{2}, w_{1}:w_{2}, m)}) ) \tag{13} $$

Average Pooling:
Reversing the average means we allocate back to each cell the value of the average. Therefore each cell receive the value of $\frac {average} {\#cells}$, i.e.:
$$ dA^{[l]} _{(c, h, w, m)} = \frac{\partial Cost}{\partial f(A^{[l]})} \frac{\partial f(A^{[l]})}{\partial A^{[l]}} = dA^{[l]} _{(c, h, w, m)} + \frac {1} {f^{[l]}_h \times f^{[l]}_w} \times \sum_{i=1} ^ {f^{[l]}_h} \sum_{j=1} ^ {f^{[l]}_w} \operatorname{devectorized} (dA^{[l+1]} _{(c, i, j, m)}) \tag{14} $$  
  
Convolutional layer:  
We can now continue the derivation following the chain rule:  
$$ dZ^{[l]} = \frac{\partial Cost}{\partial A^{[l]}} \frac{\partial A^{[l]}} {\partial Z^{[l]}} = dA^{[l]} * g'(Z^{[l]}) $$
With $ dA^{[l]} $ being the pooling gradient matrix, of dimension $A^{[l]conv]}$ given we use the Convolutional layer output as input for pooling backpropagation.
Given equation (1), it is easy to derive dW as: 
$$ dW^{[l]} _{(c, k, h, w)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial W^{[l]} _{(k, h, w)}} = \frac {1} {m} \times \sum _{i=1} ^{n^{[l]}_H} \sum_{j=1} ^ {n^{[l]}_W} dZ ^{[l]} _{(c, i, j, m)} \times \operatorname{pad} (A^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w, m)}) \tag{15}$$

Indeed, for a specific filter c, the value of the kth kernel ith row and jth column of the gradients of W is a specific value of the previous output, that we have many times because of the parameter sharing specificity of convolution. We do use this specific value of $W^{[l]}_{(c, k, i, j)}$ $n^{[l]}_H \times n^{[l]}_W$ times, and this for each $dZ^{[l]} _{(c, i, j, m)}$. Note also that $k \in [1, K^{[l]}]$, $i \in [1, f^{[l]}_h]$, $j \in [1, f^{[l]}_w]$.  
The same reasoning can be applied to compute $db^{[l]}$, which is given by:
$$ db^{[l]} _{(c)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial b^{[l]} _{(c)}} = \frac {1} {m} \times \sum _{i=1} ^{n^{[l]}_H} \sum_{j=1} ^ {n^{[l]}_W} dZ ^{[l]} _{(c, i, j, m)} \tag{16}$$

Note that if the next layer is also a convolutional layer, e.g. an architecture of conv -> conv, the calculation of $dA^{[l]}$ becomes:
$$ dA^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w, m)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial A^{[l-1]} _{(k, s(i-1)+h, s(j-1)+w, m)}} = \sum_{c=1} ^ {n^{[l]}_C} \sum _{i=1} ^{n^{[l]}_H} \sum_{j=1} ^ {n^{[l]}_W}  W^{[l]} _{(c, k, h, w)} \times dZ ^{[l]} _{(c, i, j, m)}$$  

Simplifying the indexing to: $dA^{[l-1]} _{(k, i', j', m)}$, which means $i' = s(i-1) + h$ and $j' = s(i-1) + w$, we find that $i = \frac {i'-h} {s} + 1$, $j = \frac {j'-w} {s} + 1$. However, i and j cannot be below 1 as they correspond to the minimum index of the output matrix $dZ^{[l]}$, and it also cannot go beyond its size. In other words, using Python indexing i and j are both floored at 0 and are capped at $n^{[l]}_H - 1$ and $n^{[l]}_W - 1$ respectively. Therefore, $\forall h \in [1, f^{[l]}_h]$, $\forall w \in [1, f^{[l]}_w]$:  
$$ dA^{[l-1]} _{(k, i', j', m)} = \frac{\partial Cost}{\partial Z^{[l]}} \frac{\partial Z^{[l]}} {\partial A^{[l-1]} _{(k, i', j', m)}} = \sum_{c=1} ^ {n^{[l]}_C} \sum _{i=\max (1, \lfloor \frac {i'-h} {s} + 1 \rfloor)} ^{\min (i', n^{[l]}_H)} \sum_{j= \max (1, \lfloor \frac {j'-w} {s} + 1 \rfloor)} ^ {\min (j', n^{[l]}_W)} W^{[l]} _{(c, k, h, w)} \times dZ ^{[l]} _{(c, i, j, m)} \tag{17} $$  

We finally unpad $dA^{[l-1]} _{(k, i', j', m)}$ by the appropriate padding value to come back to the original input matrix size, so that we can carry calculating gradients through (14), (15) and (16).

In [16]:
# Helper function to loop over gradient (dW, dZ) size to calculate dW and db
# obs = False for dW as we will already loop through each observation via the slicing function on A
# Whereas obs = True for db as we don't use the slicing function as seen in the equation
def gradient_loop(dX, layer, type='dW'):
    
    if type == 'dW':
        channel, _, height, width = dX[str(layer)].shape
        for c in range(channel):
            for h in range(height):
                for w in range(width):
                    yield c, h, w

    elif type == 'dZ':
        channel, height, width, observation = dX[str(layer)].shape
        for m in range(observation):
            for c in range(channel):
                for h in range(height):
                    for w in range(width):
                        yield c, h, w, m
    else:
        raise ValueError("Type needs to be either 'dW' or 'dZ'")

In [17]:
def convolutional_backward_propagation(A, Z, W, dA, dZ, dW, architecture, filter_size, padding, stride):

    # Works similar to normal backprop for fully connected layer
    # We need to compute the derivative of the last layer separately, as it is a non-generic formula
    # 1e-8 is added for numeric stability

    L = len(architecture)-1
    s = stride
    f = filter_size
    m = A['0'].shape[3]

    dAL = -Y / (A[str(L)] + 1e-8) + (1-Y) / ((1-A[str(L)]) + 1e-8)
    dZ[str(L)] = dAL * backward_activation_function(activations[L], Z[str(L)])
    dW[str(L)] = 1/m * np.dot(dZ[str(L)], A[str(L-1)].T)
    db[str(L)] = 1/m * np.sum(dZ[str(L)], axis=1, keepdims=True)

    # We can now compute the generalized backward propagation
    for l in range(L-1, 0, -1):
    
    # ___________________________________ Calculate dA ___________________________________________
        # Note that dA is computed differently in function of the previous layer (l+1), but is indifferent of the current layer.
        # It is therefore important to compute dA before hand so that we can pass on to the next gradient in the chain.
        if architecture[l+1] == 'fc':
            dA[str(l)] = np.dot(W[str(l+1)].T, dZ[str(l+1)])

        elif architecture[l+1] == 'maxpool':

            for slice, i, j, k, o in slicing(dA, A, l, architecture, filter_size, padding, stride, 'backward'):
                v1 = i * s[l]
                v2 = i * s[l] + f[l]
                h1 = j * s[l]
                h2 = j * s[l] + f[l]

                # The way it works: we take the previous layer gradient dA[l+1].
                # Each ith row, jth column, which represents the number which we will allocate to the following gradient dA[l] slice.
                # The tip is that the h, w of the hth row and wth column of the following gradient dA[l] is of the same coordinate of the maximum number
                # of A[l] e.g. the current output.
                # We therefore needs to create a mask which is a boolean matrix of dimension fxf where we have 1 if A[l][h, w] is the maximum of the current
                # operating slice, 0 otherwise.
                # We multiply this mask to the value of dA[l+1][i, j] so that we have a slice containing the dA[l+1][i, j] located where the maximum of A[l] was.
                # The slice is then fully allocated to dA[l].
                mask = (slice == np.max(slice))
                dA[str(l)][k, v1:v2, h1:h2, o] += mask * vectorization(dA[str(l+1)], A, architecture, l+1, 'matrix')[k, i, j, o]

        elif architecture[l+1] == 'avgpool':
            for slice, i, j, k, o in slicing(dA, A, l, architecture, filter_size, padding, stride, 'backward'):
                v1 = i * s[l]
                v2 = i * s[l] + f[l]
                h1 = j * s[l]
                h2 = j * s[l] + f[l]

                # The way it works: we take the previous layer gradient dA[l+1].
                # Each ith row, jth column, represents the average of the following gradient dA[l] slice.
                # Therefore, for each i, j, we allocate the value of dA[l+1] of the ith row and jth column to the corresponding slice on dA[l]
                # And we adjust it by 1/(fxf) to make a proportional allocation.
                # Note that the slicing function already iterate up to dA[l+1] height and width, so we can directly use the slice, i, j, k and o
                # taken from the slicing function.
                dA[str(l)][k, v1:v2, h1:h2, o] += vectorization(dA[str(l+1)], A, architecture, l+1, 'matrix')[k, i, j, o] / np.prod(slice.shape)

        elif architecture[l+1] == 'conv':
            _, n_h, n_w, _ = dA[str(l+1)].shape
            
            for c, h, w, o in gradient_loop(dA, l, type='dZ'):
                for k, i, j in gradient_loop(W, l+1, type='dW'):
                    # The value of dZ[l+1] needs to be clamped within its dimension
                    # Note that the stride index is l+1, given it is associated with the convolution layer, which is (l+1)
                    z_h = max(0, min( int((h-i)/s[l+1] + 1), n_h-1))
                    z_w = max(0, min( int((h-j)/s[l+1] + 1), n_w-1))
                    print("Current Layer:", l)
                    print("For dA:", c, h, w, o, k)
                    print("For dZ and W", i, j, z_h, z_w)
                    dA[str(l)][c, h, w, o] += W[str(l+1)][k, c, i, j] * dZ[str(l+1)][k, z_h, z_w, o]


    # ___________________________________ Calculate dZ, db, dW ___________________________________________
        # Note there is no dZ, dW nor db for Pooling layers
        # The fully connected layer is build very similar to the vanilla neural network
        if architecture[l] == 'fc':

            dZ[str(l)] = dA[str(l)] * backward_activation_function(activations[l], Z[str(l)])
            # Need vectorization if A[l-1] is a matrix, when changing from layer pool/conv to fc
            dW[str(l)] = 1/m * np.dot(dZ[str(l)], vectorization(A[str(l-1)], A, architecture, l, 'vector').T)
            db[str(l)] = 1/m * np.sum(dZ[str(l)], axis=1, keepdims=True)    

        elif (architecture[l] == 'conv'):
            check = 0
            # Need devectorization if the previous layer was 'fc'. Note that A is indexed at layer l but will be a vector if the previous layer is 'fc'.
            dZ[str(l)] = vectorization(dA[str(l)], A, architecture, l, 'matrix') * backward_activation_function(activations[l], Z[str(l)])

            for c, h, w in gradient_loop(dW, l, type='dW'):
                for slice, i, j, k, o in slicing(A, dA, l, architecture, filter_size, padding, stride, 'forward'):
                    check += 1
                    dW[str(l)][c, k, h, w] += np.sum(dZ[str(l)][c, i, j, o] * slice) / m

            for c, h, w, o in gradient_loop(dZ, l, type='dZ'):
                db[str(l)][c] += dZ[str(l)][c, h, w, o] / m
            print(l, check)
    return dW, db

In [18]:
dW, db = convolutional_backward_propagation(A, Z, W, dA, dZ, dW, architecture, filter_size, padding, stride)

2
For dA: 3 4 3 6 10
For dZ and W 2 4 3 1
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 3 0 2 5
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 3 1 2 4
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 3 2 2 3
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 3 3 2 2
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 3 4 2 1
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 4 0 1 5
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 4 1 1 4
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 4 2 1 3
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 4 3 1 2
Current Layer: 2
For dA: 3 4 3 6 10
For dZ and W 4 4 1 1
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and W 0 0 5 5
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and W 0 1 5 4
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and W 0 2 5 3
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and W 0 3 5 2
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and W 0 4 5 1
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and W 1 0 4 5
Current Layer: 2
For dA: 3 4 3 6 11
For dZ and

In [ ]:
W, b = update_parameters(len(architecture), W, b, dW, db, learning_rate=0.01)

print(W)
print(b)

In [ ]:
# Predict using W, b of the previous trained model
def predict(X, W, b, nb_layer, activations):

    A = {}
    A["0"] = X

    # We only need one forward propagation
    # We must include Z so that the forward propagation function doesn't return a tuple
    # However, Z is replaced with "_" as it will be unused
    _, A = convolutional_forward_propagation(A, Z, W, b, architecture, activations, filter_size, nb_kernel, padding, stride)
    Yhat = np.where(A[str(nb_layer)] <= 0.5, 0, 1)

    return Yhat

In [ ]:
# The Accuracy function
def model_accuracy(Yhat, Y):
    # Simply compare the predicted result vs real result
    # If it is the same, 1, if not 0. We then take the average
    error = np.where(Yhat == Y, 1, 0)    
    accuracy = np.mean(error)

    return accuracy

In [ ]:
# Check if the prediction function works well
result = predict(X, weight, bias, nb_layer, activations)
accuracy = model_accuracy(result, Y)

# Tuning back the results from one-hot encoding to categorical values
# This will allow us to have a better comparison when comparing with the picture
Y = np.where(Y == 0, first, second)
result = np.where(result == 0, first, second)

print("Accuracy of the model: {0:.2f}%".format(accuracy*100))

## VI. LeNet-5 Full Model
___
Now that we have implemented all the necessary function, we will put everything together to create a LeNet-5 Convolutional Neural Network.

In [ ]:
def convolutional_neural_network(X, Y, architecture, filter_size, nb_kernel, padding, stride, epoch=10, optimization=None, learning_rate=0.01, momentum_beta=0.9, rms_beta=0.999, show_cost=False):
    
    L = len(architecture) - 1
    costs = []
    A = {}
    A['0'] = X

    layer_shape = get_layer_shape(architecture, filter_size, nb_kernel, padding, stride, A)
    A, Z, W, b, dA, dZ, dW, db = initialize_model(architecture, filter_size, nb_kernel, padding, stride, A)

    for _ in range(epoch):
        Z, A = convolutional_forward_propagation(A, Z, W, b, architecture, activations, filter_size, nb_kernel, padding, stride)
        cost = compute_cost(A[str(L)], Y)
        costs.append(cost)
        dW, db = convolutional_backward_propagation(A, Z, W, dA, dZ, dW, architecture, filter_size, padding, stride)

        if optimization == None:
            W, b = update_parameters(L, W, b, dW, db, learning_rate)
        elif optimization == "momentum":
            W, b = momentum(L, W, b, dW, db, Vw, Vb, momentum_beta, learning_rate)
        elif optimization == "rmsprop":
            W, b = rmsprop(L, W, b, dW, db, Ew, Eb, i+1, rms_beta, learning_rate)
        elif optimization == "adam":
            W, b = adam(L, W, b, dW, db, Vw, Vb, Ew, Eb, i+1, momentum_beta, rms_beta, learning_rate)

    if show_cost == True:
        # Display the Cost/#Iteration plot
        plt.plot(costs)
        plt.ylabel("Cost")
        plt.xlabel("# Iteration")
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

    return W, b, costs